# Computational Theory Assessment

In [2]:
import numpy as np

## <Strong>Problem 1 </strong>: Binary Words and Operations

### Parity 
**Parity** is defined in the [Secure Hash Standard (FIPS 180-4, § 4.1.1)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

$$
\text{Parity}(x, y, z) = x \oplus y \oplus z
$$

where ⊕ shows the **bitwise XOR** operation.

For each bit position, the result is 1 if an **odd number** of x, y and z contain a 1 in that position, otherwise 0.  
Parity therefore acts as an **odd-bit detector**, ensuring that even a single bit change in the inputs alters the output.

**Simply Said:** Do I have an odd number of 1s(INPUT 1) or an even number of 1s(INPUT 0)?

**Why XOR is used**
- **XOR** stands for *exclusive OR*. 
- It's a logical operation that compares two bits and outputs **1 if they are different** and **0 if they are the same**.
- XOR is **fast**, **branch-free**, and supported directly in Hardware.  
- It implements addition mod 2 at the bit level.  

**Alternative forms in research**

Some implementations describe *Parity* differently:

| Formulation | Description | Why I Didn't Use|
| :-- | :-- | :-- |
| `(x + y + z) % 2` | Modulo-2 sum - Simplistic show of odd/even nature by summing bits and taking the remainder mod 2. |  You’d need to apply to each bit - expensive for 32-bit words. |
| `bin(x ^ y ^ z).count("1") % 2`  | Bit Count Method - parity of all bits within one number| Very slow, string conversion, not bitwise.|


**Truth Table (for 1-bit inputs)**

| x | y | z | Parity |
|:--:|:--:|:--:|:--:|
| 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 1 |
| 0 | 1 | 0 | 1 |
| 0 | 1 | 1 | 0 |
| 1 | 0 | 0 | 1 |
| 1 | 0 | 1 | 0 |
| 1 | 1 | 0 | 0 |
| 1 | 1 | 1 | 1 |

In [3]:
# Platform-defined integer type with 32 bits without sign
    # Uses np.uint32() : https://numpy.org/devdocs/user/basics.types.html
def parity(x: np.uint32, y: np.uint32, z: np.uint32):
    """
    Return the parity (XOR) of three 32-bit numbers.

    Each bit of the result is 1 if an odd number of the inputs
    have a 1 in that position, otherwise 0.

    Parameters:
        x (uint32): First 32-bit integer.
        y (uint32): Second 32-bit integer.
        z (uint32): Third 32-bit integer.

    Returns:
        np.uint32: The XOR (parity) of the three input values.
    """
    return np.uint32(x ^ y ^ z)


In [4]:
# Test values
x = np.uint32(0b10101010)
y = np.uint32(0b11001100)
z = np.uint32(0b01111000)

# Show original inputs
print("x =", np.binary_repr(x, 8))
print("y =", np.binary_repr(y, 8))
print("z =", np.binary_repr(z, 8))

# Compute Parity
result = parity(x, y, z)

# Show result in binary
# Uses np.binary_repr(): https://numpy.org/doc/2.1/reference/generated/numpy.binary_repr.html
print("Parity(x, y, z) =", np.binary_repr(result, 8))

x = 10101010
y = 11001100
z = 01111000
Parity(x, y, z) = 00011110


### Choose(x, y, z)

**Choose** is defined in the [Secure Hash Standard (FIPS 180-4, § 4.1.2)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

$$
\text{Ch}(x, y, z) = (x \land y) \oplus (\lnot x \land z)
$$

Where:  
- `∧` is the **bitwise AND** operation,  
- `⊕` is the **bitwise XOR**,  
- `¬` (or `~` in code) is the **bitwise NOT** (flips every bit).  

**Objective**  
Choose uses `x` as a **bit-selector** - for every bit position:  
- if bit from x = 1, take the corresponding bit from y;  
- if bit from x = 0, take it from z.  

So x acts like a 32-bit “key” that chooses between y and z.


**Simply said:** **Choose** picks bits from `y` or `z` depending on whether `x`’s bit is 1 or 0.


**Alternative forms and interpretations**

| Formulation | Description | Why I Didn't Use|
|:--|:--|:--|
| `(x & y) \| (~x & z)` | OR instead of XOR | Non-Uniformity within problem set  |
| `z ^ (x & (y ^ z))` | Algebraic | Doesn't Match Spec, Not Great Readability |

**Why this method (`np.uint32((x & y) ^ (~x & z))`)?**  
- It exactly follows the standard equation.  
- **np.uint32** confines results to 32 bits (no overflow).  
- Bitwise operations are hardware-level fast and branch-free — each bit is processed independently.  


**Example (1-bit truth table)**  

| x | y | z | Choose(x,y,z) |
|:-:|:-:|:-:|:-------------:|
| 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 1 |
| 0 | 1 | 0 | 0 |
| 0 | 1 | 1 | 1 |
| 1 | 0 | 0 | 0 |
| 1 | 0 | 1 | 0 |
| 1 | 1 | 0 | 1 |
| 1 | 1 | 1 | 1 |


In [14]:
def choose(x: np.uint32, y: np.uint32, z: np.uint32):
    """
    Return the result of choosing bits from y and z based on x.
    
    For each bit position, if the bit in x is 1, take the bit from y,
    otherwise take the bit from z.

    bitwise NOT operator (~) flips each bit: 0 becomes 1, and 1 becomes 0.
    bitwise AND operator (&) compares each bit of two numbers and returns 1 if both bits are 1, otherwise returns 0.
    bitwise XOR operator (^) compares each bit of two numbers and returns 1 if the bits are different, otherwise returns 0.

    Parameters:
        x (uint32): The mask.
        y (uint32): The first number.
        z (uint32): The second number.
    """
    return np.uint32((x & y) ^ (~x & z))


In [15]:
# quick tests for Choose
print(choose(np.uint32(0), np.uint32(0), np.uint32(0)))   # expect 0
print(choose(np.uint32(0), np.uint32(0), np.uint32(1)))   # expect 1
print(choose(np.uint32(1), np.uint32(1), np.uint32(0)))   # expect 1
print(choose(np.uint32(1), np.uint32(1), np.uint32(1)))   # expect 1


0
1
1
1


### Maj(x, y, z)

Defined in the [Secure Hash Standard (FIPS 180-4, § 4.1.2)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

$$
\text{Maj}(x, y, z) = (x \land y) \oplus (x \land z) \oplus (y \land z)
$$

This function outputs 1 for each bit position where **two or more** of the corresponding bits
in *x*, *y*, and *z* are 1 — that makes “majority” make sense.

Ensures strong diffusion (small input changes cause large, unpredictable output changes).

#### Example
| x | y | z | Maj(x,y,z) |
|:-:|:-:|:-:|:-----------:|
| 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 0 |
| 0 | 1 | 1 | 1 |
| 1 | 0 | 1 | 1 |
| 1 | 1 | 0 | 1 |
| 1 | 1 | 1 | 1 |




In [7]:
def Maj(x: np.uint32, y: np.uint32, z: np.uint32):
    """
    Compute the majority function.

    Maj(x, y, z) = (x & y) ^ (x & z) ^ (y & z)

    Parameters
    ----------
    x, y, z : np.uint32
        32-bit unsigned integers.

    Returns
    -------
    np.uint32
        The bitwise majority of x, y, and z.
    """
    return (x & y) ^ (x & z) ^ (y & z)

In [8]:
# Simple Testing for Majority Function
x = np.uint32(0b1010)
y = np.uint32(0b1100)
z = np.uint32(0b0110)

print(np.binary_repr(Maj(x, y, z), 4)) # expect 1110


1110


In [9]:
# Rotate-right operation for 32-bit words - Helper function for Sigma functions
def ROTR(x: np.uint32, n: int) -> np.uint32:
    """Rotate-right operation for 32-bit words."""
    return np.uint32((x >> n) | (x << np.uint32(32 - n)))

### Σ₀(x) — Sigma Zero

Defined in the [Secure Hash Standard (FIPS 180-4, §4.1.2)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

$$
\Sigma_0^{\{256\}}(x) = \text{ROTR}^2(x) \oplus \text{ROTR}^{13}(x) \oplus \text{ROTR}^{22}(x)
$$

This function rotates a 32-bit word `x` right by 2, 13, and 22 bits and XORs the results.
It helps spread (diffuse) bits of `x` across multiple positions in the SHA-256 state,
ensuring that small input changes cause large, unpredictable output changes.

| Operation | Description |
|------------|-------------|
| ROTR²(x)  | Right rotation by 2 bits |
| ROTR¹³(x) | Right rotation by 13 bits |
| ROTR²²(x) | Right rotation by 22 bits |

The final result is the XOR (⊕) of these three rotated words.


In [10]:
def Sigma0(x: np.uint32) -> np.uint32:
    """
    Compute the Σ₀ (Sigma zero) function used in SHA-256.

    Defined in FIPS 180-4 (Section 4.1.2) as:
        Σ₀(x) = ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x)

    Parameters
    ----------
    x : np.uint32
        32-bit unsigned integer input word.

    Returns
    -------
    np.uint32
        The resulting 32-bit word after applying Σ₀(x).
    """
    return ROTR(x, 2) ^ ROTR(x, 13) ^ ROTR(x, 22)


In [11]:
# Simple test for Sigma0
x = np.uint32(0b10010011)

# Display result in binary for readability
# Uses np.binary_repr(): https://numpy.org/doc/2.1/reference/generated/numpy.binary_repr.html
print("Original Number:", np.binary_repr(x, 32))
print("Sigma0 Result  :", np.binary_repr(Sigma0(x), 32))


Original Number: 00000000000000000000000010010011
Sigma0 Result  : 11000100100110100100110000100100


### Σ₁(x) — Sigma One

Defined in the [Secure Hash Standard (FIPS 180-4, §4.1.2)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

$$
\Sigma_1^{\{256\}}(x) = \text{ROTR}^6(x) \oplus \text{ROTR}^{11}(x) \oplus \text{ROTR}^{25}(x)
$$

This function performs three **bitwise right rotations** of a 32-bit word `x`
by 6, 11, and 25 bits, then XORs the results together.

Like Σ₀(x), it contributes to **diffusion** within the SHA-256 algorithm —
ensuring that small input changes propagate unpredictably throughout the hash state.

#### Behaviour Summary
| Operation | Description |
|------------|--------------|
| ROTR⁶(x)  | Circular right rotation by 6 bits |
| ROTR¹¹(x) | Circular right rotation by 11 bits |
| ROTR²⁵(x) | Circular right rotation by 25 bits |

The final output is the XOR (⊕) of these three rotated words.


In [12]:
def Sigma1(x: np.uint32) -> np.uint32:
    """
    Compute the Σ₁ (Sigma one) function.

    This function performs three right-rotations of a 32-bit word `x` by
    6, 11, and 25 bits, and returns the bitwise XOR of these results:

        Σ₁(x) = ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x)

    Parameters
    ----------
    x : np.uint32
        32-bit unsigned integer input word.

    Returns
    -------
    np.uint32
        The resulting 32-bit word after applying Σ₁(x).
    """
    return ROTR(x, 6) ^ ROTR(x, 11) ^ ROTR(x, 25)


In [13]:
# Simple test for Sigma1
x = np.uint32(0b10010011)

# Display the result in binary for readability
# Uses np.binary_repr(): https://numpy.org/doc/2.1/reference/generated/numpy.binary_repr.html
print(np.binary_repr(Sigma1(x), 32))


01011110011000000100100110000010


## <strong>Problem 2 </strong>: Fractional Parts of Cube Roots

## <strong>Problem 3 </strong>: Padding

## <strong>Problem 4 </strong>: Hashes

## <strong>Problem 5 </strong>: Passwords